<a href="https://colab.research.google.com/github/Rouhollah/text_generator_illiad/blob/main/Illiad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
import datetime
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("TensorFlow version: ", tf.__version__)

Found GPU at: /device:GPU:0
TensorFlow version:  2.12.0


In [4]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length}    bytes'.format(name=fn, length=len(uploaded[fn])))

Saving Iliad_v3.txt to Iliad_v3.txt
User uploaded file "Iliad_v3.txt" with length 886809    bytes


In [5]:
import os
path = os.getcwd()
text = open(path + '/Iliad_v3.txt',  'rb').read().decode(encoding='utf-8')
print("Text is {} characters long".format(len(text)))

Text is 886809 characters long


In [6]:
words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
print("Text is {} words long".format(len(words)))

Text is 153260 words long


In [7]:
print(text[:100])

  achilles' wrath, to greece the direful spring
  of woes unnumber'd, heavenly goddess, sing!
  that


In [10]:
vocab = sorted(set(text))
print ('There are {} unique characters'.format(len(vocab)))
char2int = {c:i for i, c in enumerate(vocab)}
int2char = np.array(vocab)
print('Vector:\n')
for char,_ in zip(char2int, range(len(vocab))):
    print(' {:4s}: {:3d},'.format(repr(char), char2int[char]))

There are 34 unique characters
Vector:

 '\n':   0,
 ' ' :   1,
 '!' :   2,
 "'" :   3,
 ',' :   4,
 '-' :   5,
 '.' :   6,
 '?' :   7,
 'a' :   8,
 'b' :   9,
 'c' :  10,
 'd' :  11,
 'e' :  12,
 'f' :  13,
 'g' :  14,
 'h' :  15,
 'i' :  16,
 'j' :  17,
 'k' :  18,
 'l' :  19,
 'm' :  20,
 'n' :  21,
 'o' :  22,
 'p' :  23,
 'q' :  24,
 'r' :  25,
 's' :  26,
 't' :  27,
 'u' :  28,
 'v' :  29,
 'w' :  30,
 'x' :  31,
 'y' :  32,
 'z' :  33,


In [14]:
text_as_int = np.array([char2int[ch] for ch in text], dtype=np.int32)
print ('{}\n mapped to integers:\n {}'.format(repr(text[:100]), text_as_int[:100]))

"  achilles' wrath, to greece the direful spring\n  of woes unnumber'd, heavenly goddess, sing!\n  that"
 mapped to integers:
 [ 1  1  8 10 15 16 19 19 12 26  3  1 30 25  8 27 15  4  1 27 22  1 14 25
 12 12 10 12  1 27 15 12  1 11 16 25 12 13 28 19  1 26 23 25 16 21 14  0
  1  1 22 13  1 30 22 12 26  1 28 21 21 28 20  9 12 25  3 11  4  1 15 12
  8 29 12 21 19 32  1 14 22 11 11 12 26 26  4  1 26 16 21 14  2  0  1  1
 27 15  8 27]


In [25]:
tr_text = text_as_int[:704000]
val_text = text_as_int[704000:]
print(text_as_int.shape, tr_text.shape, val_text.shape)


886809
(886809,) (704000,) (182809,)
(704000, 182809)


In [26]:
batch_size = 64
buffer_size = 10000
embedding_dim = 256
epochs = 50
seq_length = 200
examples_per_epoch = len(text)//seq_length
#lr = 0.001 #will use default for Adam optimizer
rnn_units = 1024
vocab_size = len(vocab)

In [27]:
tr_char_dataset = tf.data.Dataset.from_tensor_slices(tr_text)
val_char_dataset = tf.data.Dataset.from_tensor_slices(val_text)
tr_sequences = tr_char_dataset.batch(seq_length+1, drop_remainder=True)
val_sequences = val_char_dataset.batch(seq_length+1, drop_remainder=True)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text
tr_dataset = tr_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
val_dataset = val_sequences.map(split_input_target).shuffle(buffer_size).batch(batch_size, drop_remainder=True)
print(tr_dataset, val_dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int32, name=None), TensorSpec(shape=(64, 200), dtype=tf.int32, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(64, 200), dtype=tf.int32, name=None), TensorSpec(shape=(64, 200), dtype=tf.int32, name=None))>


In [28]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
     model = tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size, embedding_dim,
     batch_input_shape=[batch_size, None]),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.LSTM(rnn_units,
     return_sequences=True,
     stateful=True,
     recurrent_initializer='glorot_uniform'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.LSTM(rnn_units,
     return_sequences=True,
     stateful=True,
     recurrent_initializer='glorot_uniform'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(vocab_size)
     ])
     return model

model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=batch_size)

In [29]:
model.summary()
for input_example_batch, target_example_batch in tr_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape)
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels,    logits, from_logits=True)
example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Loss:      ", example_batch_loss.numpy().mean())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           8704      
                                                                 
 dropout (Dropout)           (64, None, 256)           0         
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dropout_1 (Dropout)         (64, None, 1024)          0         
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dropout_2 (Dropout)         (64, None, 1024)          0         
                                                                 
 dense (Dense)               (64, None, 34)            3

In [30]:
optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss=loss)
patience = 10
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience)

In [31]:
checkpoint_dir = './checkpoints'+ datetime.datetime.now().strftime("_%Y.%m.%d-%H:%M:%S")
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)
history = model.fit(tr_dataset, epochs=epochs, callbacks=[checkpoint_callback, early_stop] , validation_data=val_dataset)
print ("Training stopped as there was no improvement after {} epochs".format(patience))

Epoch 1/50
54/54 [==============================] - 27s 382ms/step - loss: 2.9997 - val_loss: 2.7491
Epoch 2/50
54/54 [==============================] - 22s 373ms/step - loss: 2.5464 - val_loss: 2.2999
Epoch 3/50
54/54 [==============================] - 22s 386ms/step - loss: 2.1712 - val_loss: 2.0360
Epoch 4/50
54/54 [==============================] - 22s 379ms/step - loss: 1.9423 - val_loss: 1.8350
Epoch 5/50
54/54 [==============================] - 22s 381ms/step - loss: 1.7625 - val_loss: 1.6897
Epoch 6/50
54/54 [==============================] - 22s 393ms/step - loss: 1.6193 - val_loss: 1.5664
Epoch 7/50
54/54 [==============================] - 22s 389ms/step - loss: 1.5110 - val_loss: 1.4804
Epoch 8/50
54/54 [==============================] - 23s 405ms/step - loss: 1.4275 - val_loss: 1.4140
Epoch 9/50
54/54 [==============================] - 23s 395ms/step - loss: 1.3622 - val_loss: 1.3657
Epoch 10/50
54/54 [==============================] - 23s 393ms/step - loss: 1.3091 - val_lo

In [33]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
def generate_text(model, start_string):

    print('Generating with seed: "' + start_string + '"')

    num_generate = 1000
    input_eval = [char2int[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 1.0
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions,      num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated.append(int2char[predicted_id])
    return (start_string + ''.join(text_generated))
print(generate_text(model, start_string="good boy"))

Generating with seed: "good boy"
good boyon,
  partike the land,
  and roll'd the gift his gulfy tounds along.
  bold apapatas'orms, his monarch fall,
  this day when deep review, and light with breast along.
  to seize his son sho took he wings its way. 
  encouraged helen as imperial foe,
  and trembling, turn'd the light!
  and thus vain points the rice the warrior wide 
  his shield his coursers yet the chief of troy
  the unrust  she rose 
  and mark'd their gods, and many a leader thrown,
  inspired, with large that all the fields are won 
  the walls iachaea's bain for ships it shine 
  the nurse ettended with his sacred breast,
  whom chief unnatived, the brave eugheston,
  depart it fall, but born  this all the race 
  that, time such argive manning swords with fire.
  from mars he lifts his brazen armour view,
  and as the danger at the bow away.
  rich cat of foes, and be no more and the rest assace 
   divine confesed! thy meges might survey,
  and trembling smiles again th